# Cálculo de armónicos esféricos utilizando computación cuántica.   
## TFE: Computación cuántica aplciada al análisis espacial en electroencefalografía   
## Master universitario en computación cuántica (UNIR)   
### Francisco Vidal Requejo   
### Antonio José Ruz Hervás   


In [6]:
%pip install pennylane > o

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pennylane as qml
from pennylane import numpy as np
import math
import sympy as sp

In [3]:
nCubits = 4

In [78]:
dev = qml.device ("default.qubit", wires = nCubits)

@qml.qnode(dev)
def CircuitoCoseno(theta, nCubits = 3):
  """
  Implementar un circuito cuántico con una puerta de ratación sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar en una rotación sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  for i in range(nCubits):
    qml.RY(theta, wires = i)
  return qml.probs(wires=range(nCubits))

def CircuitoSeno(theta, nCubits=3):
  """
  Implementar un circuito cuántico con una puerta de rotación de pi/2 - theta sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar como una rotación pi/2 - theta sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  return CircuitoCoseno((math.pi/2) - theta, nCubits)

def ExpPhi(phi, m):
  """
  Obtener el valor en coordenadas cartesianas de un número complejo en forma polar de la manera e^{im} a través de dos circuitos cuánticos de un único cúbit utilizando la igualdad e^{im} = cos(im) + i·sen(im) .

  Parámetros:
    phi: el ángulo base
    m: valor a multiplicar al ángulo.

  Retorno:
    Una tupla con el valor real e imagino del número complejo en coordenadas cartesianas.
  """
  C = CircuitoCoseno(phi * m, 1)
  S = CircuitoSeno(phi * m, 1)
  return (C[0] - C[1], S[0] - S[1])

def ProductosPotencias(theta, nCubits):
  """
  Obtener una matriz con la evaluación de los términos cos^k(theta)·sen^l(theta) con l+k = n.
  
  Parámetros:
    theta: el ángulo a aplicar a las funciones trigonométricas.
    nCubits: cantida de cúbits a utilizar, esto es el n = k+l.

  Retorno:
    Matriz con los terminos cos^k(theta)·sen^l(theta) con l+k = n.
  """
  C = CircuitoCoseno(theta, nCubits)
  S = CircuitoSeno(theta, nCubits)
  cos, sen = sp.symbols("cos sen")

def ExtraerTerminosBinomio(probsArray, nCubits = 3):
  """
  Extrae el valor los terminos de la expansión de la potencia de binomios (cos{theta}+bsen{theta})^n a partir de la matriz de probalidades obtenida de un circuito cuántico que aplica la misma rotación a toodos los cúbits.

  Parámetros:
    probArray: el array de probabilidades.
    nCubits: cantidad de cúbits del circuito del que se ha obtenido e array
  
  Retorno:
    Array de tuplas en las que el primer término es un simbolo de sympy con la expresión del témino binomial y el segundo es el valor.
  """
  cos, sen = sp.symbols("cos sen")
  res = []
  for i in range(nCubits +1):
    potenciaCoseno = cos**((nCubits - i)* 2) #como es un valor de problabilidad se multiplica por 2 cada potencia
    potenciaSeno = sen**(i * 2)
    res.append((potenciaCoseno * potenciaSeno, probsArray[2**(i) -1]))
  print(res)
  return res

def PotenciasCoseno(theta, terminosBinomio, nCubits = 3):
  """
  Obener el valor del coseno a la 'n' potencia para el ángulo 'theta'. El resultado es una lista con todas las potencias del coseno de 0 a nCubits
  """
  cos, sen = sp.symbols("cos sen")
  c = cos**2
  s = sen**2
  arr = []
  for i in range(nCubits):  #Expansión de los distintos terminos (c+s)^k(c-s)^l con k+l = nCubits.
    potenciaSuma = (c + s)**(nCubits-i - 1)
    potenciaResta = (c - s)**(i + 1)
    productoPotencias = potenciaSuma * potenciaResta
    arr.append(productoPotencias.expand())
  res = []
  for potencia, i in zip(arr, range(nCubits)): #Sustitución de los valores obtenidos del cicuito cuántico en la expansión calculada
    for termino in terminosBinomio:
      potencia.subs(termino[0], termino[1])
    res.append((cos**i, potencia))
  return res

In [65]:
#c = cos^2(theta/2)
#s = sen^2(theta/2)
cos, sen = sp.symbols("cos sen")
#c, s = sp.symbols("c s")
c = cos**2
s = sen**2
arr = []
for i in range(nCubits):
  potenciaSuma = (c + s)**(nCubits-i - 1)
  potenciaResta = (c - s)**(i + 1)
  productoPotencias = potenciaSuma * potenciaResta
  arr.append(productoPotencias.expand())
arr[0].subs(cos**6*sen**2, 1)

cos**8 - 2*cos**2*sen**6 - sen**8 + 2

In [79]:
theta = math.pi/5
C = CircuitoCoseno(theta, 4)
S = CircuitoSeno(theta, 4)

termC = ExtraerTerminosBinomio(C, 4)
termS = ExtraerTerminosBinomio(S, 4)

arrPotencias = PotenciasCoseno(theta, termC,4)

print(termC)


[(cos**8, tensor(0.6693459, requires_grad=True)), (cos**6*sen**2, tensor(0.07066473, requires_grad=True)), (cos**4*sen**4, tensor(0.00746027, requires_grad=True)), (cos**2*sen**6, tensor(0.0007876, requires_grad=True)), (sen**8, tensor(8.31493604e-05, requires_grad=True))]
[(cos**8, tensor(0.3972346, requires_grad=True)), (cos**6*sen**2, tensor(0.10312853, requires_grad=True)), (cos**4*sen**4, tensor(0.02677384, requires_grad=True)), (cos**2*sen**6, tensor(0.00695092, requires_grad=True)), (sen**8, tensor(0.00180457, requires_grad=True))]
[(cos**8, tensor(0.6693459, requires_grad=True)), (cos**6*sen**2, tensor(0.07066473, requires_grad=True)), (cos**4*sen**4, tensor(0.00746027, requires_grad=True)), (cos**2*sen**6, tensor(0.0007876, requires_grad=True)), (sen**8, tensor(8.31493604e-05, requires_grad=True))]


In [81]:
math.sin(theta)**4

0.11936437851565787